# Project: Artificial Intelligence for Land Use Conflict Analysis (AI4LUCA)

## Identification of land use dynamics and conflicts in Arauca, Casanare, and Meta from 2000 to 2022

This notebook corresponds to the **data preprocessing stage**.  

1. Loads spatial datasets for each year and department  
2. Harmonizes coordinate reference systems  
3. Standardizes attribute tables  
4. Generates clean and consistent outputs to be used in:
   - Land use change analysis
   - Conflict mapping  

Its main objective is to prepare land use / land cover datasets from multiple years for subsequent **land use conflict analysis**.

**Study area**  
- Arauca  
- Casanare  
- Meta  

**Temporal coverage**
- 2000  
- 2010  
- 2020  

### Data  

https://experience.arcgis.com/experience/568ddab184334f6b81a04d2fe9aac262/page/Datos-Abiertos-Geogr%C3%A1ficos-/  

- Corine Land Cover Colombia 2000-2002 (**cover2000**): https://www.colombiaenmapas.gov.co/?e=-91.07859485351392,-7.914089216137852,-57.41648547852286,17.909127410706898,4686&b=igac&u=0&t=43&servicio=878  
- Corine Land Cover Colombia 2010-2012 (**cover2010**): https://www.colombiaenmapas.gov.co/?e=-91.07859485351392,-7.914089216137852,-57.41648547852286,17.909127410706898,4686&b=igac&u=0&t=43&servicio=880  
- Corine Land Cover Colombia 2022 (**cover2022**): https://ideamcol-my.sharepoint.com/personal/odbeltran_ideam_gov_co/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fodbeltran%5Fideam%5Fgov%5Fco%2FDocuments%2FCarpetas%5FSEIA%2FGIG%2FCapasgeoAlt%2FCobertura%5Ftierra%5F100K%5Fperiodo%5F2022%5Flimite%5Fadministrativo%2Ezip&parent=%2Fpersonal%2Fodbeltran%5Fideam%5Fgov%5Fco%2FDocuments%2FCarpetas%5FSEIA%2FGIG%2FCapasgeoAlt&ga=1  
- Colombia Departments: https://www.colombiaenmapas.gov.co/?e=-91.07859485351392,-7.914089216137852,-57.41648547852286,17.909127410706898,4686&b=igac&u=0&t=29&servicio=609  


### Import libraries

In [1]:
import geopandas as gpd
import pandas as pd
import glob
import os
import shapely.wkb 

### Load data

Departments

In [21]:
# Load department boundaries
departments=gpd.read_file("data/departments/Departamento.shp")

ara, casa, meta = [departments[departments["DeNombre"] == d]
                    for d in ["Arauca", "Casanare", "Meta"]] # Select departments

# Project to EPSG:9377 and save as GeoPackage
for name, gdf in {"ara": ara, "casa": casa, "meta": meta}.items():
    if gdf.crs != "EPSG:9377":
        gdf = gdf.to_crs(epsg=9377)
    gdf = gdf[["geometry"]]   # keep only geometry column
    gdf.to_file(f"./data/departments/{name}.gpkg", driver="GPKG")

Land cover

In [13]:
# Paths to land cover shapefiles
covers = {
    "cover2000": "./data/cover/cover_2000/cover_2000.shp",
    "cover2010": "./data/cover/cover_2010/cover_2010.shp",
    "cover2022": "./data/cover/cover_2022/cover_2022.shp"}

# Project to EPSG:9377 and save as GeoPackage
for name, path in covers.items():
    gdf = gpd.read_file(path)
    if gdf.crs != "EPSG:9377":
        gdf = gdf.to_crs(epsg=9377)
    gdf.to_file(f"./data/cover/gpkg/{name}.gpkg", driver="GPKG")

### Clip land cover by department 

In [22]:
# Department and cover identifiers
departments = ["ara", "casa", "meta"]
covers = ["cover2000", "cover2010", "cover2022"]

# Clip and save
for dep in departments:
    dep_gdf = gpd.read_file(f"./data/departments/{dep}.gpkg")

    for cover in covers:
        cover_gdf = gpd.read_file(f"./data/cover/gpkg/{cover}.gpkg")

        # Spatial intersection
        clipped = gpd.overlay(cover_gdf, dep_gdf, how="intersection")

        # Save output
        out_path = f"./data/cover/departments/{dep}_{cover}.gpkg"
        clipped.to_file(out_path, driver="GPKG")

        print(f"Saved: {out_path}")

Saved: ./data/cover/departments/ara_cover2000.gpkg
Saved: ./data/cover/departments/ara_cover2010.gpkg
Saved: ./data/cover/departments/ara_cover2022.gpkg
Saved: ./data/cover/departments/casa_cover2000.gpkg
Saved: ./data/cover/departments/casa_cover2010.gpkg
Saved: ./data/cover/departments/casa_cover2022.gpkg
Saved: ./data/cover/departments/meta_cover2000.gpkg
Saved: ./data/cover/departments/meta_cover2010.gpkg
Saved: ./data/cover/departments/meta_cover2022.gpkg


### Get the level 2 of land cover classes

Load departments land cover

In [4]:
# Find and read them into a dictionary
covers_dep_files = glob.glob(os.path.join("./data/cover/departments", "*.gpkg"))
covers_dep = {os.path.basename(f).replace(".gpkg", ""): gpd.read_file(f) for f in covers_dep_files}

Create a new column with Level 2 classes, and extract its respective code class. The new datasets will include three columns: Level 2 classes, Level 2 code, and geometry.

In [6]:
# Define the mapping from level 2 codes to their descriptions
level_2_mapping = {
    "1.1": "Urban",
    "1.2": "Industrial and commercial",
    "1.3": "Mine, dump and construction",
    "1.4": "Artificial non-agricultural vegetated areas",
    "2.1": "Arable land",
    "2.2": "Permanent crops",
    "2.3": "Pastures",
    "2.4": "Heterogeneous agricultural areas",
    "3.1": "Forest",
    "3.2": "Shrub or herbaceous vegetation",
    "3.3": "Little vegetated areas",
    "4.1": "Wetlands",
    "5.1": "Water bodies"
}

def map_to_level_2(level_3_code):
    """
    Handles inputs like "3.2.3" (dot format) AND "323" or "31122" (no dot format).
    """
    code_str = str(level_3_code).strip()
    
    if "." in code_str:
        # Handle "3.2.3" -> "3.2"
        parts = code_str.split(".")
        if len(parts) >= 2:
            level_2_code = f"{parts[0]}.{parts[1]}"
        else:
            level_2_code = parts[0]
    else:
        # Handle "323" -> "3.2" or "31122" -> "3.1"
        if len(code_str) >= 2:
            level_2_code = f"{code_str[0]}.{code_str[1]}"
        else:
            level_2_code = code_str

    level_2_class = level_2_mapping.get(level_2_code, "Unknown")
    return f"{level_2_code} {level_2_class}"

def extract_level2_code(level2_str):
    # Extracts '1.1' from '1.1 Urban' and converts to '11'
    code = level2_str.split()[0]
    return code.replace('.', '')

# Output folder
output_folder = "./data/cover/coverl2"

# Process datasets
datasets = {
    "ara_cover2000l2": (covers_dep['ara_cover2000'], "LEYENDA"),
    "ara_cover2010l2": (covers_dep['ara_cover2010'], "LEYENDA3N"),
    "ara_cover2022l2": (covers_dep['ara_cover2022'], "leyenda"),
    "casa_cover2000l2": (covers_dep['casa_cover2000'], "LEYENDA"),
    "casa_cover2010l2": (covers_dep['casa_cover2010'], "LEYENDA3N"),
    "casa_cover2022l2": (covers_dep['casa_cover2022'], "leyenda"),
    "meta_cover2000l2": (covers_dep['meta_cover2000'], "LEYENDA"),
    "meta_cover2010l2": (covers_dep['meta_cover2010'], "LEYENDA3N"),
    "meta_cover2022l2": (covers_dep['meta_cover2022'], "leyenda")
}

# New dictionary to store processed Level 2 datasets
lc = {}

for dataset_name, (gdf, column_name) in datasets.items():
    # Force 2D geometry (PolygonZ -> Polygon)
    if gdf.has_z.any():
        gdf["geometry"] = gdf["geometry"].apply(
            lambda geom: shapely.wkb.loads(shapely.wkb.dumps(geom, output_dimension=2)))
        
    # Add a new column for level 2 categories
    gdf["level2"] = gdf[column_name].apply(map_to_level_2)

    # Add level2_code column
    gdf["l2_code"] = gdf["level2"].apply(extract_level2_code)
    
    # Keep the level2 name and code, and geometry columns
    gdf = gdf[["level2", "l2_code","geometry"]]
    # Store in dictionary
    lc[dataset_name] = gdf

    # Save the updated dataset
    output_path = os.path.join(output_folder, f"{dataset_name}.gpkg")
    gdf.to_file(output_path, driver="GPKG", layer=dataset_name)
    print(f"Processed {dataset_name} and saved to {output_path}")

Processed ara_cover2000l2 and saved to ./data/cover/coverl2\ara_cover2000l2.gpkg
Processed ara_cover2010l2 and saved to ./data/cover/coverl2\ara_cover2010l2.gpkg
Processed ara_cover2022l2 and saved to ./data/cover/coverl2\ara_cover2022l2.gpkg
Processed casa_cover2000l2 and saved to ./data/cover/coverl2\casa_cover2000l2.gpkg
Processed casa_cover2010l2 and saved to ./data/cover/coverl2\casa_cover2010l2.gpkg
Processed casa_cover2022l2 and saved to ./data/cover/coverl2\casa_cover2022l2.gpkg
Processed meta_cover2000l2 and saved to ./data/cover/coverl2\meta_cover2000l2.gpkg
Processed meta_cover2010l2 and saved to ./data/cover/coverl2\meta_cover2010l2.gpkg
Processed meta_cover2022l2 and saved to ./data/cover/coverl2\meta_cover2022l2.gpkg


In [7]:
lc["ara_cover2000l2"].head()

,level2,l2_code,geometry
0,3.2 Shrub or herbaceous vegetation,32,"POLYGON ((5102592.293 2249694.850, 5102551.324..."
1,3.2 Shrub or herbaceous vegetation,32,"POLYGON ((5101603.687 2243259.080, 5101673.883..."
2,3.2 Shrub or herbaceous vegetation,32,"POLYGON ((5101613.222 2245350.430, 5101586.973..."
3,3.2 Shrub or herbaceous vegetation,32,"POLYGON ((5101238.220 2247849.283, 5101274.432..."
4,3.2 Shrub or herbaceous vegetation,32,"POLYGON ((5108706.356 2253956.586, 5108702.068..."


### Land Use Vocation

In [8]:
# Arauca 
ara_voc= gpd.read_file("./data/vocation/ara_voc.gpkg")
ara_voc= ara_voc.to_crs(epsg=9377)
out_path = f"./data/vocation/ara_voc.gpkg"
ara_voc.to_file(out_path, driver="GPKG")
# Casanare
casa_voc= gpd.read_file("../data/vocation/casa_voc.gpkg")
casa_voc= casa_voc.to_crs(epsg=9377)
out_path = f"./data/vocation/casa_voc.gpkg"
casa_voc.to_file(out_path, driver="GPKG")
# Meta
meta_voc= gpd.read_file("./data/vocation/meta_voc.gpkg")
meta_voc= meta_voc.to_crs(epsg=9377)
out_path = f"./data/vocation/meta_voc.gpkg"
meta_voc.to_file(out_path, driver="GPKG")

### Intersection: **Land Cover** and **Land Use Vocation**  


Arauca

In [12]:
# Perform the intersection
ara_lcvoc_2000 = gpd.overlay(lc['ara_cover2000l2'], ara_voc, how="intersection")

# Save the result to a file
output_path = "./data/lc_voc/ara_lcvoc_2000.gpkg"
ara_lcvoc_2000.to_file(output_path, driver="GPKG")

c:\Users\VICTUS\anaconda3\envs\osgeo-env-v1\lib\site-packages\ipykernel_launcher.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 73 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  


In [13]:
# Perform the intersection
ara_lcvoc_2010 = gpd.overlay(lc['ara_cover2010l2'], ara_voc, how="intersection")

# Save the result to a file
output_path = "./data/lc_voc/ara_lcvoc_2010.gpkg"
ara_lcvoc_2010.to_file(output_path, driver="GPKG")

c:\Users\VICTUS\anaconda3\envs\osgeo-env-v1\lib\site-packages\ipykernel_launcher.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 272 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  


In [10]:
# Perform the intersection
ara_lcvoc_2022 = gpd.overlay(lc['ara_cover2022l2'], ara_voc, how="intersection")

# Save the result to a file
output_path = "./data/lc_voc/ara_lcvoc_2022.gpkg"
ara_lcvoc_2022.to_file(output_path, driver="GPKG")

Casanare

In [15]:
# Perform the intersection
casa_lcvoc_2000 = gpd.overlay(lc['casa_cover2000l2'], casa_voc, how="intersection")

# Save the result to a file
output_path = "./data/lc_voc/casa_lcvoc_2000.gpkg"
casa_lcvoc_2000.to_file(output_path, driver="GPKG")

In [16]:
# Perform the intersection
casa_lcvoc_2010 = gpd.overlay(lc['casa_cover2010l2'], casa_voc, how="intersection")

# Save the result to a file
output_path = "./data/lc_voc/casa_lcvoc_2010.gpkg"
casa_lcvoc_2010.to_file(output_path, driver="GPKG")

c:\Users\VICTUS\anaconda3\envs\osgeo-env-v1\lib\site-packages\ipykernel_launcher.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 780 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  


In [11]:
# Perform the intersection
casa_lcvoc_2022 = gpd.overlay(lc['casa_cover2022l2'], casa_voc, how="intersection")

# Save the result to a file
output_path = "./data/lc_voc/casa_lcvoc_2022.gpkg"
casa_lcvoc_2022.to_file(output_path, driver="GPKG")

c:\Users\VICTUS\anaconda3\envs\osgeo-env-v1\lib\site-packages\ipykernel_launcher.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 33 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  


Meta

In [18]:
# Perform the intersection
meta_lcvoc_2000 = gpd.overlay(lc['meta_cover2000l2'], meta_voc, how="intersection")

# Save the result to a file
output_path = "./data/lc_voc/meta_lcvoc_2000.gpkg"
meta_lcvoc_2000.to_file(output_path, driver="GPKG")

In [19]:
# Perform the intersection
meta_lcvoc_2010 = gpd.overlay(lc['meta_cover2010l2'], meta_voc, how="intersection")

# Save the result to a file
output_path = "./data/lc_voc/meta_lcvoc_2010.gpkg"
meta_lcvoc_2010.to_file(output_path, driver="GPKG")

c:\Users\VICTUS\anaconda3\envs\osgeo-env-v1\lib\site-packages\ipykernel_launcher.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 211 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  


In [12]:
# Perform the intersection
meta_lcvoc_2022 = gpd.overlay(lc['meta_cover2022l2'], meta_voc, how="intersection")

# Save the result to a file
output_path = "./data/lc_voc/meta_lcvoc_2022.gpkg"
meta_lcvoc_2022.to_file(output_path, driver="GPKG")

c:\Users\VICTUS\anaconda3\envs\osgeo-env-v1\lib\site-packages\ipykernel_launcher.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 11 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  


Level 1

In [ ]:
# Input and output folders
covers_dir = "./data/cover/coverl2"
output_folder = "./data/cover/coverl1"
os.makedirs(output_folder, exist_ok=True)

# Mapping dictionary
level_1_mapping = {
    "1": "Artificial surfaces",
    "2": "Agricultural areas",
    "3": "Forest and seminatural areas",
    "4": "Wetlands",
    "5": "Water bodies"
}

# Find all GPKG files
covers_dep_files = glob.glob(os.path.join(covers_dir, "*.gpkg"))

# Dictionary to store new GeoDataFrames
lc1 = {}

# Process each file
for file_path in covers_dep_files:
    # Read file
    gdf = gpd.read_file(file_path)
    
    # Extract first digit and map to level1
    gdf["level1"] = gdf["l2_code"].astype(str).str[0].map(level_1_mapping)
    
    # Keep only desired columns
    gdf = gdf[["level1", "level2", "geometry"]]
    
    # Store in dictionary
    name = os.path.basename(file_path).replace("l2.gpkg", "")
    lc1[name] = gdf
    
    # Save to new folder
    out_path = os.path.join(output_folder, f"{name}l1.gpkg")
    gdf.to_file(out_path, driver="GPKG")